In [ ]:
! pip install kaggle
! mkdir /content/.kaggle
! cp kaggle.json /content/kaggle.json
! chmod 600 /content/kaggle.json


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
cp: 'kaggle.json' and '/content/kaggle.json' are the same file


In [ ]:
! kaggle competitions download -c g2net-detecting-continuous-gravitational-waves

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.8/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.8/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import h5py
import timm
import torch
import torch.nn as nn
import torchaudio
import torchvision.transforms as TF

from tqdm.auto import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

device = torch.device('cuda')

# Train metadata
di = '../input/g2net-detecting-continuous-gravitational-waves'
df = pd.read_csv(di + '/train_labels.csv')
df = df[df.target >= 0]  # Remove 3 unknowns (target = -1)

In [ ]:
transforms_time_mask = nn.Sequential(
                torchaudio.transforms.TimeMasking(time_mask_param=10),
            )

transforms_freq_mask = nn.Sequential(
                torchaudio.transforms.FrequencyMasking(freq_mask_param=10),
            )

flip_rate = 0.0 # probability of applying the horizontal flip and vertical flip 
fre_shift_rate = 0.0 # probability of applying the vertical shift

time_mask_num = 0 # number of time masking
freq_mask_num = 0 # number of frequency masking

In [ ]:
def img_clipping(x, p=3, standard=256):
    point = np.percentile(x, 100-p)
    imgo = ((standard//2)*(x-x.min())/(point-x.min()))
    imgo = np.clip(imgo, 0, standard)
    return imgo.astype(np.float32)

In [ ]:
class Dataset(torch.utils.data.Dataset):
    """
    dataset = Dataset(data_type, df)

    img, y = dataset[i]
      img (np.float32): 2 x 360 x 128
      y (np.float32): label 0 or 1
    """
    def __init__(self, data_type, df, tfms=False):
        self.data_type = data_type
        self.df = df
        self.tfms = tfms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, i):
        """
        i (int): get ith data
        """
        r = self.df.iloc[i]
        y = np.float32(r.target)
        file_id = r.id

        img = np.empty((2, 360, 128), dtype=np.float32)

        filename = '%s/%s/%s.hdf5' % (di, self.data_type, file_id)
        with h5py.File(filename, 'r') as f:
            g = f[file_id]
            
            for ch, s in enumerate(['H1', 'L1']):
                a = g[s]['SFTs'][:, :4096] * 1e22  # Fourier coefficient complex64
                p = a.real**2 + a.imag**2  # power
                #p /= np.mean(p)  # normalize
                p -= np.mean(p)
                p = np.mean(p.reshape(360, 128, 32), axis=2)  # compress 4096 -> 128
                p = img_clipping(p)
                img[ch] = p

        if self.tfms:
            if np.random.rand() <= flip_rate: # horizontal flip
                img = np.flip(img, axis=1).copy()
            if np.random.rand() <= flip_rate: # vertical flip
                img = np.flip(img, axis=2).copy()
            if np.random.rand() <= fre_shift_rate: # vertical shift
                img = np.roll(img, np.random.randint(low=0, high=img.shape[1]), axis=1)
            
            img = torch.from_numpy(img)

            for _ in range(time_mask_num): # tima masking
                img = transforms_time_mask(img)
            for _ in range(freq_mask_num): # frequency masking
                img = transforms_freq_mask(img)
        
        else:
            img = torch.from_numpy(img)
                
        return img, y

In [ ]:
class PatchEmbed(nn.Module):
  def __init__(self,img_size,patch_size,in_chans=1,embed_dims=330):
    super(PatchEmbed,self).__init__()
    self.img_size = img_size
    self.patch_size = patch_size
    self.n_patches = (img_size // patch_size)**2
    self.proj = nn.Conv2d(in_chans,embed_dims,kernel_size=patch_size,stride=patch_size)

  def forward(self,x):
    x = self.proj(x)
    x = x.flatten(2)
    x = x.transpose(1,2)
    return x

class Attention(nn.Module):
  def __init__(self,dim,n_heads=10,qkv_bias=True,attn_p=0.,proj_p=0.):
    super(Attention,self).__init__()
    self.n_heads = n_heads
    self.dim = dim
    self.head_dim = dim // n_heads
    self.scale = self.head_dim**-0.5
    self.qkv = nn.Linear(dim,dim*3,bias=qkv_bias)
    self.attn_drop = nn.Dropout(attn_p)
    self.proj = nn.Linear(dim,dim)
    self.proj_drop = nn.Dropout(proj_p)

  def forward(self,x):
    n_samples,n_tokens,dim = x.shape
    if dim != self.dim:
      raise ValueError
    qkv = self.qkv(x)
    qkv = qkv.reshape(n_samples,n_tokens,3,self.n_heads,self.head_dim)
    qkv = qkv.permute(2,0,3,1,4)
    q,k,v = qkv[0],qkv[1],qkv[2]
    k_t = k.transpose(-2,-1)
    dp = (q @ k_t)*self.scale
    attn = dp.softmax(dim=-1)
    weighted_avg = attn @ v
    weighted_avg = weighted_avg.transpose(1,2)
    weighted_avg = weighted_avg.flatten(2)
    x = self.proj(weighted_avg)
    x = self.proj_drop(x)
    return x
  
class MLP(nn.Module):
  def __init__(self,in_features,hidden_features,out_features,p=0.):
    super(MLP,self).__init__()
    self.gelu = nn.GELU()
    self.sigmoid = nn.Sigmoid()
    self.fc1 = nn.Linear(in_features,hidden_features)
    self.fc2 = nn.Linear(hidden_features,hidden_features)
    self.fc3 = nn.Linear(hidden_features,hidden_features)
    self.fc4 = nn.Linear(hidden_features,out_features)
    self.drop = nn.Dropout(p)

  def forward(self,x):
    x = self.fc1(x)
    x = self.gelu(x)
    #x = self.drop(x)
    x = self.fc2(x)
    x = self.gelu(x)
    #x = self.drop(x)
    x = self.fc3(x)
    x = self.gelu(x)
    #x = self.drop(x)
    x = self.fc4(x)
    x = self.sigmoid(x)
    #x = self.drop(x)
    return x

class Block(nn.Module):
  def __init__(self,dim,n_heads,mlp_ratio=4,qkv_bias=True,p=0.,attn_p=0.):
    super(Block,self).__init__()
    self.norm1 = nn.LayerNorm(dim,eps=1e-9,)
    self.attn = Attention(dim=dim,n_heads=n_heads,qkv_bias=qkv_bias,attn_p=attn_p,proj_p=p)
    self.norm2 = nn.LayerNorm(dim,eps=1e-9)
    hidden_features = int(dim*mlp_ratio)
    self.mlp = MLP(in_features=dim,hidden_features=hidden_features,out_features=dim,p=p)

  def forward(self,x):
    x = x + self.attn(self.norm1(x))
    x = x + self.mlp(self.norm2(x))
    return x


class VisionTransformer(nn.Module):
  def __init__(self,img_size=128,patch_size=10,in_chans=1,n_classes=1,embed_dim=64,depth=12,n_heads=10,mlp_ratio=4,qkv_bias=True,p=0.,attn_p=0.):
    super(VisionTransformer,self).__init__()
    self.patch_embed = PatchEmbed(img_size=img_size,patch_size=patch_size,in_chans=in_chans,embed_dims=330)
    self.cls_token = nn.Parameter(torch.zeros(1,1,embed_dim))
    self.pos_embed = nn.Parameter(torch.zeros(1,1+self.patch_embed.n_patches,embed_dim))
    self.pos_drop = nn.Dropout(p=p)
    self.blocks = nn.ModuleList([Block(dim=embed_dim,n_heads=n_heads,qkv_bias=qkv_bias,p=p,attn_p=p)for j in range(depth)])
    self.norm = nn.LayerNorm(embed_dim,eps=1e-6)
    self.head = nn.Linear(embed_dim, n_classes)

  def forward(self,x):
    n_samples = x.shape[0]
    x = self.patch_embed(x)
    cls_token = self.cls_token.expand(n_samples,-1,-1)
    x = torch.cat((cls_token,x),dim=1)
    x = x + self.pos_embed
    x = self.pos_drop(x)
    for block in self.blocks:
      x = block(x)
    x = self.norm(x)
    cls_token_final = x[:,0]
    x = self.head(cls_token_final)
    return x



In [ ]:
epochs = 25
batch_size = 16
num_workers = 2
dataset_train = Dataset('train', df, tfms=True)
training_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, num_workers=num_workers, pin_memory=True, shuffle=True, drop_last=True)

In [ ]:
#custom_config = {
#       "img_size": 165,
#        "in_chans": 1,
#        "patch_size": 10,
#        "embed_dim": 330,
#        "depth": 12,
#        "n_heads": 10,
#        "qkv_bias": True,
#        "mlp_ratio": 4,
#}

model = VisionTransformer()
learning_rate = 0.002
optimiser = torch.optim.AdamW(model.parameters(),lr = learning_rate,weight_decay=1e-6)
criterion = nn.BCELoss()

for epoch in range(epochs):
  for i,(X,y) in enumerate(training_loader):
    y_predicted = model(X)
    loss = criterion(y_predicted,y)
    loss.backward()
    optimiser.step()
    print('iteration: ',i)
  if epoch == epochs:
    print('Training Complete')

torch.save(model,'/content/model')